In [1]:
import torch
from torch import nn


class FullyConnectedModule(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int):
        super().__init__()

        # input layer
        module_list = [nn.Linear(input_size, hidden_size), nn.ReLU()]
        # hidden layers
        for _ in range(n_hidden_layers):
            module_list.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU()])
        # output layer
        module_list.append(nn.Linear(hidden_size, output_size))

        self.sequential = nn.Sequential(*module_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x of shape: batch_size x n_timesteps_in
        # output of shape batch_size x n_timesteps_out
        return self.sequential(x)


# test that network works as intended
network = FullyConnectedModule(input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)
x = torch.rand(20, 5)
network(x).shape

torch.Size([20, 2])

In [2]:
# Writing Pytorch Forecasting Model
from typing import Dict

from pytorch_forecasting.models import BaseModel


class PFFullyConnectedModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input)

        # rescale predictions into target space
        prediction = self.transform_output(prediction, target_scale=x["target_scale"])

        # We need to return a dictionary that at least contains the prediction
        # The parameter can be directly forwarded from the input.
        # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
        return self.to_network_output(prediction=prediction)

/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# Read Dataset
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
test_data = pd.read_csv('data/dados_ponto_recife.csv')
time_p_group=32
total_timesteps = np.arange(0,test_data.shape[0])
test_data['timestep'] = total_timesteps%time_p_group
test_data['group'] = total_timesteps//time_p_group
training_cutoff = "YYYY-MM-DD"

In [4]:
from pytorch_forecasting import TimeSeriesDataSet

# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    test_data[lambda x: x.time <= training_cutoff],
    group_ids=["group"],
    target="tp",
    time_idx="timestep",
    min_encoder_length=30,
    max_encoder_length=30,
    min_prediction_length=2,
    max_prediction_length=2,
    time_varying_unknown_reals=["tp"],
)
dataset.get_parameters()

/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/pytorch_forecasting/data/timeseries.py:1281: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__group': 23010}]
  warnings.warn(


{'time_idx': 'timestep',
 'target': 'tp',
 'group_ids': ['group'],
 'weight': None,
 'max_encoder_length': 30,
 'min_encoder_length': 30,
 'min_prediction_idx': 0,
 'min_prediction_length': 2,
 'max_prediction_length': 2,
 'static_categoricals': [],
 'static_reals': [],
 'time_varying_known_categoricals': [],
 'time_varying_known_reals': [],
 'time_varying_unknown_categoricals': [],
 'time_varying_unknown_reals': ['tp'],
 'variable_groups': {},
 'constant_fill_strategy': {},
 'allow_missing_timesteps': False,
 'lags': {},
 'add_relative_time_idx': False,
 'add_target_scales': False,
 'add_encoder_length': False,
 'target_normalizer': EncoderNormalizer(
 	method='standard',
 	center=True,
 	max_length=None,
 	transformation=None,
 	method_kwargs={}
 ),
 'categorical_encoders': {'__group_id__group': NaNLabelEncoder(add_nan=False, warn=True)},
 'scalers': {},
 'randomize_length': None,
 'predict_mode': False}

In [5]:
# convert the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=64)

# and load the first batch
x, y = next(iter(dataloader))
print("x =", x)
print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

x = {'encoder_cat': tensor([], size=(64, 30, 0), dtype=torch.int64), 'encoder_cont': tensor([[[-1.1034],
         [-0.8777],
         [-0.0219],
         ...,
         [ 0.3083],
         [-0.1499],
         [-1.2315]],

        [[ 2.2836],
         [ 0.0376],
         [-0.8560],
         ...,
         [-0.1154],
         [-0.5421],
         [-0.7755]],

        [[-0.6656],
         [-0.7025],
         [-0.7075],
         ...,
         [ 0.6749],
         [ 0.3587],
         [ 0.2174]],

        ...,

        [[-0.7809],
         [-0.7301],
         [-0.7042],
         ...,
         [-0.1790],
         [ 0.5042],
         [-0.3027]],

        [[-0.3272],
         [-0.0702],
         [-0.8505],
         ...,
         [ 0.2648],
         [-0.8551],
         [-0.7908]],

        [[-0.2047],
         [-0.1620],
         [ 1.9013],
         ...,
         [-0.4608],
         [-0.3825],
         [-0.5035]]]), 'encoder_target': tensor([[0.0139, 0.0266, 0.0749,  ..., 0.0935, 0.0677, 0.0067],
  

In [6]:
model = PFFullyConnectedModel.from_dataset(dataset, input_size=30, output_size=2, hidden_size=1000, n_hidden_layers=4, learning_rate=1e-3)
x, y = next(iter(dataloader))
model(x)

/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Output(prediction=tensor([[0.0227, 0.0241],
        [0.2367, 0.2423],
        [0.0653, 0.0677],
        [0.0144, 0.0150],
        [0.1215, 0.1245],
        [0.0281, 0.0301],
        [0.0991, 0.1013],
        [0.2239, 0.2306],
        [0.0947, 0.0971],
        [0.0888, 0.0955],
        [0.0503, 0.0531],
        [0.1831, 0.1872],
        [0.0883, 0.0917],
        [0.3705, 0.3762],
        [0.1139, 0.1183],
        [0.0786, 0.0826],
        [0.0332, 0.0354],
        [0.0796, 0.0850],
        [0.0092, 0.0098],
        [1.3202, 1.3486],
        [0.0177, 0.0188],
        [0.1190, 0.1227],
        [0.2087, 0.2183],
        [0.3177, 0.3245],
        [0.7613, 0.7740],
        [0.1023, 0.1052],
        [0.1390, 0.1428],
        [0.4958, 0.5138],
        [0.3739, 0.3884],
        [0.2677, 0.2817],
        [0.0743, 0.0776],
        [0.3849, 0.3918],
        [0.1129, 0.1157],
        [0.0881, 0.0905],
        [0.2541, 0.2583],
        [0.0382, 0.0396],
        [0.1606, 0.1655],
        [0.0040, 0.0

In [7]:
model.hparams

"hidden_size":                 1000
"input_size":                  30
"learning_rate":               0.001
"log_gradient_flow":           False
"log_interval":                -1
"log_val_interval":            -1
"logging_metrics":             ModuleList()
"monotone_constaints":         {}
"n_hidden_layers":             4
"optimizer":                   Ranger
"optimizer_params":            None
"output_size":                 2
"output_transformer":          EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation=None,
	method_kwargs={}
)
"reduce_on_plateau_min_lr":    1e-05
"reduce_on_plateau_patience":  1000
"reduce_on_plateau_reduction": 2.0
"weight_decay":                0.0

In [8]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="lr-Ranger", min_delta=1e-4, patience=1, verbose=True, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="auto",  # run on CPU, if on multiple GPUs, use strategy="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=10000,  # 30 batches per epoch
    callbacks=[lr_logger],
    # callbacks=[early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
# from lightning.pytorch.tuner import Tuner
# res = Tuner(trainer).lr_find(
#     model, train_dataloaders=dataloader, early_stop_threshold=1000.0, max_lr=0.3,
# )

In [10]:
trainer.fit(
    model, train_dataloaders=dataloader ,# val_dataloaders=val_dataloader
)

/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params | Mode 
-----------------------------------------------------------------
0 | loss            | SMAPE                | 0      | train
1 | logging_metrics | ModuleList           | 0      | train
2 | network         | FullyConnectedModule | 4.0 M  | train
-----------------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.148    Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode
/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bot

Epoch 1:   1%|  | 5/359 [00:00<00:12, 29.10it/s, v_num=24, train_loss_step=1.020, train_loss_epoch=0.941]

/home/lpa1/anaconda3/envs/weather/lib/python3.10/site-packages/lightning/pytorch/loops/training_epoch_loop.py:389: ReduceLROnPlateau conditioned on metric val_loss which is not available but strict is set to `False`. Skipping learning rate update.


Epoch 999: 100%|█| 359/359 [00:11<00:00, 30.14it/s, v_num=24, train_loss_step=0.333, train_loss_epoch=0.4

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|█| 359/359 [00:12<00:00, 29.46it/s, v_num=24, train_loss_step=0.333, train_loss_epoch=0.4


In [11]:
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss
tft = TemporalFusionTransformer.from_dataset(
    # dataset
    dataloader.dataset,
    # architecture hyperparameters
    hidden_size=36,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=6,
    # loss metric to optimize
    loss=QuantileLoss(),
    # logging frequency
    log_interval=2,
    # optimizer parameters
    learning_rate=0.03,
    # reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 68.2k


In [12]:
tft

TemporalFusionTransformer(
  	"attention_head_size":               2
  	"categorical_groups":                {}
  	"causal_attention":                  True
  	"dropout":                           0.1
  	"embedding_labels":                  {}
  	"embedding_paddings":                []
  	"embedding_sizes":                   {}
  	"hidden_continuous_size":            6
  	"hidden_continuous_sizes":           {}
  	"hidden_size":                       36
  	"learning_rate":                     0.03
  	"log_gradient_flow":                 False
  	"log_interval":                      2
  	"log_val_interval":                  2
  	"lstm_layers":                       1
  	"max_encoder_length":                30
  	"monotone_constaints":               {}
  	"optimizer":                         Ranger
  	"optimizer_params":                  None
  	"output_size":                       7
  	"output_transformer":                EncoderNormalizer(
  		method='standard',
  		center=True,
  		ma

In [29]:
o = model(x)
# x
o.prediction
# model.transform_output(o.prediction,x['target_scale'])

tensor([[ 2.3748e-02,  2.2495e-02],
        [ 9.8031e-02,  1.0646e-01],
        [ 1.0351e-01,  3.6386e-02],
        [-3.5438e-05,  6.2159e-03],
        [ 8.0782e-02,  8.4940e-02],
        [ 4.0815e-03,  4.4085e-03],
        [ 1.9856e-01,  1.7889e-01],
        [ 3.7247e-03,  4.1513e-03],
        [ 6.0195e-03,  8.8222e-03],
        [ 6.0095e-03,  3.0140e-02],
        [-3.0226e-03, -4.0457e-03],
        [ 4.4956e-01,  3.5594e-01],
        [ 5.2398e-01,  5.2683e-01],
        [ 5.0546e-01,  4.8624e-01],
        [ 1.2502e-02,  3.8954e-02],
        [ 3.1226e-03,  4.3975e-03],
        [ 8.9126e-03,  2.6478e-03],
        [ 1.0307e+00,  1.3347e+00],
        [ 8.2471e-04,  2.1327e-03],
        [ 1.0463e+01,  1.5974e+01],
        [ 1.2068e-01,  1.6947e-01],
        [ 7.3796e-02,  1.8571e-01],
        [ 5.6835e-02,  6.5052e-02],
        [ 5.7857e-01,  5.0337e-01],
        [ 2.3188e-01,  2.5130e-01],
        [ 1.1520e-01,  1.5184e-02],
        [ 1.6728e-01,  3.2459e-01],
        [ 4.3851e-02,  3.696

In [28]:
y

(tensor([[2.5272e-02, 2.2411e-02],
         [1.0014e-01, 1.1110e-01],
         [1.0300e-01, 3.6240e-02],
         [0.0000e+00, 0.0000e+00],
         [8.6784e-02, 8.2493e-02],
         [-0.0000e+00, -0.0000e+00],
         [2.0218e-01, 1.8311e-01],
         [2.8610e-03, 8.1060e-03],
         [3.3380e-03, 1.1444e-02],
         [5.2060e-03, 3.3350e-02],
         [0.0000e+00, 0.0000e+00],
         [4.4203e-01, 3.4952e-01],
         [5.0020e-01, 5.2166e-01],
         [4.9305e-01, 4.7493e-01],
         [1.4305e-02, 4.2915e-02],
         [2.8950e-03, 4.3430e-03],
         [8.5830e-03, 1.9070e-03],
         [1.1220e+00, 1.3724e+00],
         [0.0000e+00, 0.0000e+00],
         [1.0683e+01, 1.5770e+01],
         [1.1921e-01, 1.7738e-01],
         [7.5340e-02, 1.8501e-01],
         [5.1022e-02, 6.1989e-02],
         [5.9175e-01, 5.3215e-01],
         [2.9659e-01, 2.5988e-01],
         [1.1349e-01, 1.7166e-02],
         [1.6792e-01, 3.2800e-01],
         [3.1948e-02, 2.8610e-02],
         [1.5926e-